# 图像恢复高级专题III 课程实践作业

## 第一部分 神经形态视觉信号基本处理

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from util import plot_figs, load_event_npz, load_spike_npz, get_flow_viz_map

BLANK = None

# 如果你需要导入其他的包，可以写在这里
# 如果你需要定义自己的函数，可以写在这里，也可以写在调用位置附近

### 1. 神经形态视觉数据读取与表示

#### 1.1 事件数据

本部分提供的事件数据，是以 `npz` 格式存储的事件四元组 $(t, x, y, p)$ 列表。根据任务的需要，有些数据里还附带有对应的传统帧灰度图像。数据的具体格式与内容如下：

事件数据：在 `npz` 文件中的名称为 `event`，是个 `(N, 4)` 的 `np.ndarray`，数组内元素为整型类型。其中第一列是时间戳（单位：μs）；第二、三列分别是横、纵坐标（单位：像素）；第四列是极性，其中 ON 事件记为 1，OFF 事件记为 0。

传统帧数据：在 `npz` 文件中的名称为 `frame`，是个 `(N, h, w)` 的 `np.ndarray`。像素值以浮点数存储，范围是 $[0, 1]$ （若无特殊说明，本次作业中的所有传统图像帧都应当采用这样的表示方式）。数组的第一个维度表示时间。在本部分中，提供的传统帧数据的帧率都是 240 fps，第一帧的起始时间戳都是 0，事件数据和传统帧数据具有相同的空间尺寸。

> 提示：由上可知，下标为 $i$ 的传统图像对应的时间区间应该是 $[\frac{i}{240}, \frac{i+1}{240})$，时间戳处在这个范围之内的事件，就是对应到这张传统图像的事件信息。

##### 1.1.1 事件的读取

请根据上面的提示，给出从下标为 15 的传统图像开始曝光起，至下标为 19 的传统图像结束曝光止，对应的时间区域；并且根据这个时间区域，截取出对应的事件数据范围，以填补下面留为 `BLANK` 的变量（可以直接给出变量的值，也可以写出计算这个变量值的代码，下同）。

In [6]:
event, frame = load_event_npz('basketball.npz')

i, j = 15, 19
fps = 240

# TODO  请在下面填空  ↓↓↓

event_begin_timestamp = BLANK   # 单位：μs
event_end_timestamp = BLANK

event_begin_idx = BLANK
event_end_idx = BLANK

# ↑↑↑

event_clip = event[event_begin_idx:event_end_idx]
print(event_clip.shape)

##### 1.1.2 事件的表示

事件帧是一种简单直观的事件信号表示方式。顾名思义，事件的帧表示参照了传统成像原理中“帧”的概念，将一个时间窗口内的事件按像素位置累加到一张图像（即二维矩阵）上，形成一“帧”图，即

$$ F(x,y) = \sum_{x_i=x, y_i=y, t \in [t, t+\Delta t)} c_i $$

在本次作业中，$c_i$ 取事件的极性。请填补下面的空白代码，计算上一小问中截取出的事件序列所对应的事件帧。

In [ ]:
def get_event_frame(event_clip, sz):
    # TODO  请在这里实现事件帧的计算  ↓↓↓
    # 提示：使用循环遍历事件列表实现累加可能会很慢，请考虑使用 numpy, pytorch 等库的向量化操作
    pass

event_frame = get_event_frame(event_clip, sz=frame.shape[-2:])
plot_figs((frame[(i+j)//2], 'Scene'), (event_frame, 'Event Frame'))

#### 1.2 脉冲数据格式

本部分提供的脉冲数据，是以 `npz` 格式存储的脉冲平面数据。对于每个 `npz` 文件，其中的脉冲数据名为 `spike` ，是个大小为 `(N, h, w)` 的 `np.ndarray` 数组，数组内元素为无符号整型类型，第一个维度表示时间。在本部分中，脉冲数据的时间分辨率是 25 μs （即每个脉冲平面记录了 25 μs 内的脉冲触发情况，每个数组总共记录 0.25 s ）。触发了脉冲的位置记为 1，否则记为 0。

> 注意：`spike` 数据量较大，请注意可用内存大小。

请截取出下面的脉冲序列中从第 0.10 s 起至第 0.13s 止的脉冲平面，填补下面留为 `BLANK` 的变量。 

In [ ]:
spike = load_spike_npz('rotation.npz')

dt = 25

spike_begin_timestamp = 0.10 * 1e6
spike_end_timestamp = 0.13 * 1e6

# TODO  请在这里填空  ↓↓↓

spike_begin_idx = BLANK
spike_end_idx = BLANK

# ↑↑↑

spike_clip = spike[spike_begin_idx:spike_end_idx]

print(spike_clip.shape)

### 2. 事件信号重构

根据事件信号的成像原理，给定一幅传统图像起始帧，再给定这幅图像对应时刻之后的一系列“变化”（即事件），就可以重建出后续时刻的场景帧，可由下式描述：

$$ L_{t_1} = L_{t_0} + \sum_{t=t_0}^{t_1} C_i\cdot p_i $$

其中 $L_{t_0}$ 是起始帧，时间戳为 $t_0$；$L_{t_1}$ 是重建的目标帧，时间戳 $t_1$。用 $L$ 来表示意味着它们都是对数域上的数据 。$p_i$ 是事件 $i$ 的极性，ON 事件则有 $p_i = 1$ ，OFF 事件则有 $p_i = -1$ 。$C_i$ 是事件 $i$ 的触发阈值，在实际的事件相机中，正负事件的阈值可能不相等。对于本题目中的数据，$C_{+} = C_{-} = 0.09$。

接下来请你填补下面的空白代码、以及留为 `BLANK` 的变量，完成这个过程。

`basketball.npz` 文件中存储了本题目所需的事件和传统帧信息。以其中的首帧为输入帧，**下标为 10 的帧**为重建目标。观察输出结果相对输入结果的变化。


In [ ]:
# 这里的事件数据空间分辨率是 (180, 240)
# 注意：这里的图像已经预处理到对数域上了，下同
event_1, frame_1 = load_event_npz('basketball.npz')

i, j = 0, 10
image_t0_1, image_t1_1 = frame[i], frame[j]


def event_recon(event, image, C):
    # TODO  请在这里实现简单累计重构  ↓↓↓
    # 提示：使用循环遍历事件列表实现累积可能会很慢，请考虑使用 numpy, pytorch 等库的向量化操作
    pass


# TODO  请在这里填空  ↓↓↓

event_input_1 = BLANK

# ↑↑↑

image_recon_1 = event_recon(event_input_1, image_t0_1, C=0.09)
plot_figs((image_t0_1, 'Input'), (image_t1_1, 'Target'), (image_recon_1.clip(0, 1), 'Your Output'))

实际上，简单累积重建方法鲁棒性较差，如果是较为复杂的场景，或是较长的累积时长，就很难得到合理的重建效果，例如下面的例子 `building.npz` ：

In [ ]:
# 这里的事件数据空间分辨率是 (260, 346)
event_2, frame_2 = load_event_npz('building.npz')

i, j = 0, 39
image_t0_2, image_t1_2 = frame[i], frame[j]

# TODO  请在这里填空  ↓↓↓

event_input_2 = BLANK

# ↑↑↑

image_recon_2 = event_recon(event_input_2, image_t0_2, C=0.08)
plot_figs((image_t0_2, 'Input'), (image_t1_2, 'Target'), (image_recon_2.clip(0, 1), 'Your Output'))

如果你的代码在这组样例上测试的重建结果中出现了很多视觉上不清晰、不合理的内容，并不一定代表你的代码实现存在问题，而是由于简单累积重建鲁棒性较差。简要分析结果质量较差的原因，阐述可能的改进方案<small>（不要求实现，但能动手验证思路、给出测试结果更佳）</small>。

    你的回答：（双击 Markdown 内容可以编辑）

### 3 脉冲信号重构

根据脉冲信号的成像原理，给定一组脉冲序列，可以直接重建出场景帧。接下来请你填补下面的空白代码、以及留为 `BLANK` 的变量，完成这个过程。

> 提示：重构的结果需要进行伽马校正。

#### 3.1 TFP 重构

TFP 重构可由下式描述：

$$ L_{t_i} = \frac{N_{w}}{w}\cdot C, $$

其中，$L_{t_i}$ 表示 $t_i$ 时刻的重构图像；$w$ 表示时间窗口长度，对应 $[t_i-w, t_i]$ 之间发放的脉冲；$N_{w}$ 表示脉冲数目；$C$ 是重构图像中的最大值（这里是 1.0）。

请根据上式，重建出
1. `rotation` 场景中，$t_i = 0.15\ \mathrm{s}$，$w = 0.01\ \mathrm{s}$ 的重构图像。
2. `drone` 场景中，$t_i = 0.10\ \mathrm{s}$，$w = 0.01\ \mathrm{s}$ 的重构图像。

In [ ]:
def spike_recon_tfp(spike, ti, w):
    # TODO  请在这里实现 TFP 重构  ↓↓↓
    pass

spike_1 = load_spike_npz('rotation.npz')
image_recon_tfp_1 = spike_recon_tfp(spike_1, ti=0.15, w=0.01)

spike_2 = load_spike_npz('drone.npz')
image_recon_tfp_2 = spike_recon_tfp(spike_2, ti=0.10, w=0.01)

plot_figs((image_recon_tfp_1.clip(0, 1), 'Your TFP Output: rotation'), (image_recon_tfp_2.clip(0, 1), 'Your TFP Output: drone'))

调整 `w` 的值（至少尝试 5 个不同的值），观察重构结果的变化，描述你的发现。

    你的回答：（双击 Markdown 内容可以编辑）

In [ ]:
# TODO  请在下面填空  ↓↓↓

spike = BLANK   # 两个场景，任选其一即可
w_list = BLANK
result_list = []
for w in w_list:
    result_list.append(spike_recon_tfp(spike, ti=BLANK, w=w))

# ↑↑↑

plot_figs(*zip(result_list, (f'w={w}' for w in w_list)))

#### 3.2 TFI 重构

TFI 重构可由下式描述：

$$ L_{t_i} = \frac{C}{\Delta t_i}, $$

其中，$L_{t_i}$ 表示 $t_i$ 时刻的重构图像；$\Delta t_i$ 表示 $t_i$ 时刻的各像素的脉冲间隔，是在 $t_i$ 时刻后的某像素发放的第一个脉冲与 $t_i$ 时刻之前发放的最后一个脉冲的时间戳之差；$C$ 是重构的最大值（这里是 1.0）。

请根据上式，重建出 
1. `rotation` 场景中，$t_i = 0.15\ \mathrm{s}$ 的重构图像。
2. `drone` 场景中，$t_i = 0.10\ \mathrm{s}$ 的重构图像。

In [ ]:
def spike_recon_tfi(spike, ti):
    # TODO  请在这里实现 TFI 重构  ↓↓↓
    pass

image_recon_tfi_1 = spike_recon_tfp(spike_1, ti=0.15)
image_recon_tfi_2 = spike_recon_tfp(spike_2, ti=0.10)

plot_figs((image_recon_tfi_1.clip(0, 1), 'Your TFI Output: rotation'), (image_recon_tfi_2.clip(0, 1), 'Your TFI Output: drone'))

对比两种重构方法的结果，描述你的发现，并尝试总结两种方法各自适用的场景。

    你的回答：（双击 Markdown 内容可以编辑）

### 4 事件信号运动分析

对比度最大化（Contrast Maximization, CM）是事件信号运动分析的一种方法。计算反推帧的方差，找到使这个方差最大的运动参数，就可以完成光流估计、深度估计、旋转估计等任务。接下来请你实现对比度最大化的光流估计过程。先计算出光流，然后根据这个光流导出反推事件帧。

#### 4.1 光流计算

这里给出一种较为简单直接的“网格搜索”思路：

1. 预先估计场景中 $x$ 和 $y$ 两个方向上运动速度的范围 $V_x$, $V_y$，根据问题的精度要求设置一定的步长 $s$。
2. 以每个像素为中心划定网格（这里取 $5\times 5$ 的网格），在每个网格中，对 $V_x$ 和 $V_y$ 分别每隔 $s$ 进行一次采样，计算各速度对应的区块反推对比度，并记录最大的那一个，它对应的速度就是此像素对应的光流。
    - 在计算反推对比度时，会遇到反推的事件坐标不是整数的情况。这里可以采用双线性插值的方法，也可以直接取整。
3. 上一步计算出的光流是对各像素光流的一个粗估计。以这个速度为中心，以更细的粒度再次划定网格，计算各像素的光流。出于计算效率的考虑，这个细化操作只算一次即可，不需要迭代收敛。
4. 对每个区块都这样计算，就可以得到所有像素对应的光流值了。你得到的光流值应该是两个 `(96, 96)` 的矩阵，分别对应 $x$ 和 $y$ 方向的光流。

这个方法比较容易理解，缺点是效率较低。但本题对计算效率不做要求，能实现正确的功能即可。你可以根据这个思路直接实现，也可以尝试提出自己的解法。

<small> 本部分的计算时间可能较长，请耐心等待。 可以利用 `tqdm` 等工具追踪计算的进度。</small>

> 提示：在存储的数据中，事件的时间戳以 μs 为单位；而在下面的代码中，光流的单位是 px / s，要注意单位的转换

In [ ]:
# 这里的事件数据空间分辨率是 (96, 96)
event_3, _ = load_event_npz('temple.npz')

# 预设的参数（请勿修改）
v_range = 100
u_range = 10
grid_num = 10
r = 2

# TODO  请实现 flow_x 和 flow_y 的计算  ↓↓↓
# 这里给出的代码模板仅作为提示，你可以根据自己的需要随意修改
Vx = np.linspace(-v_range, v_range, grid_num)
Vy = np.linspace(-v_range, v_range, grid_num)

flow_x, flow_y = np.zeros((96, 96)), np.zeros((96, 96))
for iy in range(96):
    for ix in range(96):

        # TODO: 1. 划分网格
        pass
        # 提示：可以跳过不存在事件的网格以加快计算

        # TODO: 2. 在网格上搜索
        pass
        
        # TODO: 3. 细化速度搜索
        pass

        # 提示：上面的 2 和 3 进行的操作有相同的部分，可以考虑将其定义为函数，并在这里调用

        flow_x[iy, ix] = BLANK
        flow_y[iy, ix] = BLANK


plot_figs(
    (get_flow_viz_map(flow_x, flow_y), 'Your Output'), 
    (get_flow_viz_map(*np.meshgrid(np.linspace(-1, 1, 96), np.linspace(-1, 1, 96))), 'Flow Color Wheel')
)

#### 4.2 导出反推帧

根据上面计算出的光流，将给定事件序列中的事件全部反推回 $t_\text{ref} = 0$ 时刻的位置，得到对应的反推帧。

In [ ]:
def warp_event(event, flow_x, flow_y, t_ref=0):
    # TODO  请在这里实现反推事件计算  ↓↓↓
    pass

event_warped = warp_event(event_3, flow_x, flow_y, t_ref=0)

event_frame_original = get_event_frame(event_3, sz=(96, 96))
event_frame_warped = get_event_frame(event_warped, sz=(96, 96))

plot_figs((event_frame_original, 'Event Frame before Warping'), (event_frame_warped, 'Event Frame after Warping'))